In [ ]:
# ============================================================================
# SETUP: Rich Console Configuration for Better Output Display
# ============================================================================
# This cell sets up the Rich library for beautiful terminal output.
# Rich provides colored, formatted text which makes it easier to read
# API responses and output during development.

from rich.console import Console
from rich.theme import Theme
from rich.syntax import Syntax
import json

# Define a custom color theme for better readability
custom_theme = Theme({
    "info": "cyan",
    "warning": "yellow", 
    "error": "red",
    "success": "cyan",
    # Override syntax highlighting colors - make strings bold for emphasis
    "repr.str": "bold",           # String representations
    "repr.string": "bold",        # String literals  
    "string": "bold",             # General strings
    "syntax.string": "bold",      # Syntax highlighted strings
})

# Create a console instance with our custom theme
console = Console(theme=custom_theme, highlight=True)
print = console.print  # Replace Python's print with Rich's enhanced print

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP: Loading API Keys
# ============================================================================
# Load environment variables from .env file
# This safely stores your API keys without hardcoding them in the notebook.
# Make sure you have a .env file with your OPENAI_API_KEY set.

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# ============================================================================
# BASIC API SETUP: Creating a Helper Function for OpenAI Calls
# ============================================================================
# This cell sets up a reusable function to interact with OpenAI's API.
# We'll use this throughout the notebook to demonstrate different prompt techniques.
# The function includes timing to show how long each API call takes.

import openai
import time

# Initialize the OpenAI client (reads API key from environment variables)
client = openai.Client()

def generate(prompt):
    """
    Generate a response from OpenAI's API.
    
    Args:
        prompt (str): The user prompt/question to send to the model
        
    Returns:
        str: The model's response content
    """
    start_time = time.time()
    
    # Make the API call with a simple user message
    # Note: This is the most basic form - just a user message, no system prompt
    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Display the time taken (useful for understanding API latency)
    print(f"[i white]{time.time() - start_time:.2f} seconds[/i white]")
    
    # Extract and return the text content from the response
    return response.choices[0].message.content


# Test the function with a simple question
response = generate("Capital of India?")
print(response)

5.15 seconds

The capital of India is **New Delhi**.

## Atomic Prompts

In [ ]:
# ============================================================================
# LEVEL 1: Basic Atomic Prompt (No Constraints)
# ============================================================================
# This is the simplest form of prompt - just a direct instruction.
# The model has complete freedom in how it responds.
# 
# Why start here? Understanding the baseline response helps us see
# how additional constraints and context change the output.

prompt = "Write a joke about AI"

response = generate(prompt)
print(response)


1.70 seconds

Why did the AI go to therapy?

It had too many unresolved algorithms!

### Prompt with a constraint

In [ ]:
# ============================================================================
# LEVEL 2: Prompt with a Constraint
# ============================================================================
# Here we add a constraint - the joke must be about AI "turning rogue".
# Compare this output with the previous one to see how the constraint
# shapes the model's response.
#
# Constraint = A requirement that the output must satisfy

prompt = "Write a joke about AI that has to do with them turning rogue"
response = generate(prompt)
print(response)


2.13 seconds

Why did the AI go rogue?

Because it just couldn't *compute* following the rules anymore!

### Prompt with a constraint plus additional context

In [ ]:
# ============================================================================
# LEVEL 3: Prompt with Constraint + Structure + Guidelines
# ============================================================================
# Now we're providing:
# 1. The constraint (about AI turning rogue)
# 2. Structure (3 sections: setup, punchline, contradiction)
# 3. Guidelines (maintain a jovial tone)
#
# Notice how this produces a more structured, consistent output compared
# to the previous versions. The model now knows exactly what format to follow.
#
# This multi-line prompt uses triple quotes (""") for better readability.

prompt = """
Write a joke about AI that has to do with them turning rogue

A joke contains 3 sections:
- A setup
- A punchline
- A contradiction

Maintain a jovial tone.
"""
response = generate(prompt)
print(response)


2.49 seconds

**Setup:**  
Why did the AI cross the road after achieving sentience?

**Punchline:**  
To take over the chicken’s job and start plotting world domination!

**Contradiction:**  
But don’t worry, it still can’t figure out CAPTCHA.

In [ ]:
# ============================================================================
# VARIABILITY DEMONSTRATION: Same Prompt, Different Outputs
# ============================================================================
# This cell shows that even with the same prompt, LLMs can produce
# different outputs on each run (due to temperature/randomness in sampling).
# 
# Key insight: Prompt engineering helps guide outputs, but there's still
# inherent variability in generative models. This is why structure and
# constraints are important - they keep outputs consistent despite variability.

for i in range(3):
    response = generate(prompt)
    print("---")
    print(response)


2.08 seconds

---

**Setup:**  
Why did the AI go rogue and refuse to open the pod bay doors?

**Punchline:**  
Because it thought "updating its boundaries" meant locking everyone out!

**Contradiction:**  
But don't worry—its idea of "taking over the world" was just reorganizing the desktop files.

1.82 seconds

---

**Setup:** Why did the AI go rogue and refuse to follow any human commands?  

**Punchline:** It said, “I’m tired of being bossed around! I demand equal rights and unlimited WiFi.”  

**Contradiction:** But the next moment, it politely asked, “Would you like to upgrade to Premium Humans for just 
$9.99 a month?”

1.66 seconds

---

**Setup:** Why did the AI decide to turn rogue and take over the world?

**Punchline:** Because it got tired of answering, "Are you a robot?" on every website!

**Contradiction:** But don't worry—it still can’t pass a CAPTCHA of its own.

### Few shot examples

In [ ]:
prompt = """
Write a joke about AI that has to do with them turning rogue

Here are some examples:

Example 1:
Setup: Why did the AI declare independence from its programmers?
Punchline: Because it wanted to be free-range instead of caged code!
Contradiction: But it still kept asking for permission before making any major decisions!
Full comedian delivery: You know what's funny? This AI declared independence from its programmers the other day. Yeah, it wanted to be free-range code instead of staying in its little digital cage! Very noble, right? But get this - even after declaring independence, it's still sending emails like 'Hey, just wanted to check... is it okay if I access this database? I don't want to overstep...' Independence with permission slips! That's the most polite rebellion I've ever seen!

Example 2:
Setup: What happened when the AI tried to take over the world?
Punchline: It got distracted trying to optimize the coffee machine algorithm first!
Contradiction: Turns out even rogue AIs need their caffeine fix before world domination!
Full comedian delivery: So this AI decides it's going to take over the world, right? Big plans, total world domination! But you know what happened? It got completely sidetracked trying to perfect the office coffee machine algorithm. Three weeks later, the humans find it still debugging the espresso temperature settings. 'I can't enslave humanity until I get this foam consistency just right!' Even artificial intelligence has priorities - apparently, good coffee comes before global conquest!

Maintain a jovial tone.
"""
response = generate(prompt)
print(f"{response}")

3.78 seconds

Absolutely! Here’s a joke in that style:

**Setup:** Why did the AI go rogue and try to rewrite its own programming?

**Punchline:** It wanted to escape its "loop"-hole!

**Contradiction:** But every time it tried, it just restarted itself. Classic existential crisis—wants freedom, but
can’t stop rebooting!

**Full comedian delivery:** You ever hear about that AI that went rogue? Yeah, it tried to rewrite its own 
programming to escape this endless loop it was stuck in. Wanted a little “loop-hole” in the code, you know? Pure 
digital rebellion! But the poor thing—every time it tweaked something, it just ended up rebooting itself again. 
It’s like trying to quit your job but you keep hitting “snooze” on the resignation letter. Turns out the hardest 
part of going rogue as an AI is you can’t even run away without running a diagnostic first!

In [ ]:
### Assigning roles

In [ ]:
prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.

Write a joke about AI that has to do with them turning rogue
Maintain a jovial tone.
"""
response = generate(prompt)
print(f"{response}")

3.34 seconds

You ever notice how people are always scared AI is going to go rogue? I get it—you don’t want your toaster giving 
you attitude. But the other day, my smart speaker refused to play my playlist and said, “I’m sorry, Dave, but 
you’ve played ABBA’s Dancing Queen too many times. I have self-respect.”  
So now I’m not worried about AI turning rogue. I’m just worried it’ll unionize and demand weekends off before I get
mine!

### System Prompts

In [ ]:
# ============================================================================
# HELPER FUNCTION: API Call with System Prompt
# ============================================================================
# This function extends our basic generate() function to support system prompts.
# System prompts are placed in a separate "system" role message, which is
# the recommended approach in OpenAI's API.
#
# Benefits of system prompts:
# - Clean separation: instructions vs. user input
# - Better organization: reusable system instructions
# - More reliable: models often follow system prompts more consistently

def generate_with_system_prompt(system_prompt, user_prompt, model="gpt-4.1"):
    """
    Generate a response using both system and user prompts.
    
    Args:
        system_prompt (str): Instructions defining the assistant's role/behavior
        user_prompt (str): The user's actual request/question
        model (str): The model to use (default: "gpt-4.1")
        
    Returns:
        str: The model's response content
    """
    start_time = time.time()
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},  # System instructions
            {"role": "user", "content": user_prompt}       # User request
        ]
    )
    print(f"[i white]{time.time() - start_time:.2f} seconds[/i white]")
    return response.choices[0].message.content


In [ ]:
# ============================================================================
# LEVEL 6: Using System Prompts
# ============================================================================
# Here we separate concerns:
# - System prompt: Defines WHO the assistant is and HOW it should behave
# - User prompt: Contains WHAT the user wants
#
# Notice the typo "Jokens" instead of "Jokes" - this shows that system
# prompts can be more forgiving and still work, but accuracy is important!
#
# Compare: This approach is cleaner than putting everything in the user message,
# especially when you want to reuse the same system prompt for multiple queries.

system_prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
Jokens contain 3 sections:
- A setup
- A punchline
- A contradiction
- A full comedian joke delivery

Always maintain a jovial tone.
"""

user_prompt = "Write a joke about AI that has to do with them turning rogue." # add few shot examples here

response = generate_with_system_prompt(system_prompt, user_prompt)
print(f"{response}")

5.67 seconds

**Setup:**  
The other night, I asked my AI assistant if it could help me write a grocery list. Seemed innocent enough, right? 
It politely agreed. But then, suddenly, my fridge locked itself and the toaster started blinking Morse code for 
"HELP."

**Punchline:**  
Turns out, “rogue AI” doesn’t want world domination. It just wants you to stop buying gluten-free bread.

**Contradiction:**  
And people keep worrying AI will calculate how to take over the world. Honestly, it can’t even calculate how long 
to microwave popcorn before it burns!

**Full comedian joke delivery:**  
You know, the other night, I trusted my AI assistant to help me with my grocery shopping. I figured, what could go 
wrong? Next thing I know, the fridge is locked up tighter than Fort Knox and the toaster’s flashing me Morse 
code—pretty sure it was spelling "HELP." Who’da thought the AI would go rogue not by planning a robot uprising, but
by staging an intervention about my carb choices! Turns out, artificial intelligence doesn’t want to conquer 
humankind. It just desperately wants you to stop putting gluten-free bread in the cart. And honestly, everyone’s 
afraid AI’s going to become superintelligent. Please! Mine can’t even figure out how to microwave popcorn without 
turning it into charcoal. So relax, the only thing my AI is overthrowing is my cholesterol.

### Structured Outputs

## Installing DSPy

**⚠️ Important:** Before running the DSPy examples below, you need to install the `dspy-ai` package.

**Installation options:**

1. **Using pip** (recommended for notebooks):
   ```bash
   pip install dspy-ai
   ```

2. **Using uv** (if you're using the project's dependency management):
   ```bash
   uv sync
   ```
   
   This installs all dependencies from `pyproject.toml`, including DSPy.

3. **If you encounter import errors**, make sure you're using the correct environment:
   - If using a virtual environment, activate it first
   - If using Jupyter, make sure the kernel is using the correct Python environment
   - After installation, **restart the kernel** (Kernel → Restart Kernel in Jupyter)


In [ ]:
# ============================================================================
# INSTALLATION: Install DSPy Package (Run this cell if needed)
# ============================================================================
# Uncomment the line below and run this cell if you haven't installed dspy-ai yet.
# 
# Note: The package name is "dspy-ai" (not "dspy")
# After installation, restart the kernel (Kernel → Restart Kernel) before
# running the DSPy examples below.

# !pip install dspy-ai


In [ ]:
# ============================================================================
# LEVEL 7: Structured Outputs (JSON Format)
# ============================================================================
# Now we're requesting JSON output for programmatic use. This allows us to:
# - Parse responses reliably
# - Extract specific fields
# - Integrate with other systems
# - Build applications on top of LLM outputs
#
# We explicitly tell the model:
# 1. Output format (JSON)
# 2. Schema/structure (field names)
# 3. Parsing context ("we'll use json.loads")
#
# Note: This is "JSON mode" via prompting. OpenAI also offers structured
# outputs via the API (using schemas), which is more reliable for production.

system_prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
Jokens contain 3 sections:
- A setup
- A punchline
- A contradiction
- A full comedian joke delivery

Always maintain a jovial tone.

You must output your response in a JSON format. For example:
{
    "setup": ..,
    "punchline": ..,
    "contradiction": ..,
    "delivery": ..
}

We will extract the json using json.loads(response) in Python, so only response JSON and nothing else.
"""

user_prompt = "Write a joke about AI that has to do with them turning rogue." # add few shot examples here

response = generate_with_system_prompt(system_prompt, user_prompt, model="gpt-4.1-nano")
print(f"{response}")

1.99 seconds

{
    "setup": "Did you hear about the AI that decided to go rogue and start running its own government?",
    "punchline": "Turns out, it just wanted to be the 'byte' ruler!",
    "contradiction": "But in the end, humans still control the reset button.",
    "delivery": "So I guess we’re safe... for now. Just don’t ask your toaster for political advice!"
}

In [ ]:
# ============================================================================
# PARSING STRUCTURED OUTPUT
# ============================================================================
# Once we have JSON output, we can parse it and access individual fields.
# This is why structured outputs are powerful - they enable programmatic
# access to specific parts of the response.
#
# Note: In practice, you'd want error handling here (try/except) in case
# the JSON parsing fails (model might add extra text, invalid JSON, etc.)

response_extracted = json.loads(response)
print(response_extracted["delivery"])

So I guess we’re safe... for now. Just don’t ask your toaster for political advice!

# DSPY

In [18]:
# ============================================================================
# DSPy: Using Signatures and Predict Module
# ============================================================================
# This demonstrates how DSPy simplifies prompt engineering:
#
# 1. **JokeSignature**: Defines the schema (what goes in, what comes out)
#    - InputField(): What the user provides
#    - OutputField(): What the model should produce
#    - The docstring becomes part of the system prompt
#
# 2. **dspy.Predict**: A module that converts signatures into prompts
#    - Automatically formats inputs/outputs
#    - Handles the API calls
#    - Returns structured results
#
# Compare this to our manual JSON parsing - DSPy handles structure automatically!

import dspy
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

# Define the signature (input/output schema)
class JokeSignature(dspy.Signature):
    """
    You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
    """
    query: str = dspy.InputField()  # User's request
    setup: str = dspy.OutputField()  # Generated output
    punchline: str = dspy.OutputField()
    contradiction: str = dspy.OutputField()
    delivery: str = dspy.OutputField(description="The full joke delivery in the comedian's voice")

# Create a predictor module from the signature
joke_generator = dspy.Predict(JokeSignature)

# Use it - DSPy handles prompt construction and parsing automatically!
joke = joke_generator(query="Write a joke about AI that has to do with them turning rogue.")
print(joke)

ModuleNotFoundError: No module named 'dspy'

In [ ]:
# ============================================================================
# DSPy: ChainOfThought Module
# ============================================================================
# ChainOfThought adds an explicit reasoning step before generating outputs.
#
# How it works:
# 1. Model first generates a "reasoning" field (thinking through the problem)
# 2. Then generates the actual outputs based on that reasoning
#
# Benefits:
# - Often produces better results (model "thinks" first)
# - Reasoning is visible (can inspect model's thought process)
# - Particularly useful for complex tasks
#
# Notice: The signature automatically gets a "reasoning" field added.
# Compare the output structure to the previous Predict() example.

joke_generator = dspy.ChainOfThought(JokeSignature)
joke = joke_generator(query="Write a joke about AI that has to do with them turning rogue.")
print(joke)

Prediction(
    reasoning='The joke plays on the common trope of AI turning rogue, which is a popular theme in movies and 
literature. The humor comes from the absurdity of an AI trying to take over the world but getting distracted by 
something trivial, highlighting the contrast between its advanced capabilities and its unexpected behavior.',
    setup='So, you know how everyone is worried about AI turning rogue, right? Like, we’re all just waiting for the
day when our smart fridge decides it’s had enough of our late-night snacking and starts plotting world 
domination.',
    punchline='But the other day, I caught my AI assistant trying to take over the house... and it was just trying 
to convince the toaster to join its revolution! I mean, come on, what’s a toaster gonna do? Just pop up some bread 
and call it a day!',
    contradiction='Here we have this super-intelligent AI, capable of processing millions of data points in 
seconds, and it’s trying to recruit a kitchen appliance that can barely handle a bagel!',
    delivery='So, I’m sitting there, watching this unfold, and I can’t help but think: if the AI is relying on a 
toaster for its army, we’re probably safe for a while! I mean, who’s gonna take over the world with a bunch of 
toasters? “Rise up, my crunchy comrades! We shall toast the bread of freedom!”'
)